In [1]:
import numpy as np
import pandas as pd
from sklearn.base import clone 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RepeatedKFold, KFold, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE, SelectKBest, mutual_info_regression
import matplotlib.pyplot as plt
import time
import json
import sys
import os
# make paths above 'notebooks/' visible for local imports.
# +----------------------------------------------------------------------------+
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.processing import GatherFeatureDatasets
from src.utils import CrossValidation
from src.feature_selection import CustomRFECV
from src.feature_selection import IntrinsicFeatureSelection as ifs

# Load in the data

In [2]:
proc = GatherFeatureDatasets(is_p=True)

In [3]:
data_dir = '/uufs/chpc.utah.edu/common/home/koper-group3/alysha/magnitudes/feature_splits'
all_train_df = pd.read_csv(f'{data_dir}/p.train.csv')

In [4]:
# Use station YUF for experimenting with models because it has 1433 training examples, 
# which is ~half as much as YNR, so training should go faster. But should still be enough 
# examples for training good models. It is also one of the stations with a 
# UUSS correction, so already used for magnitude calculations and amplitudes should be good quality. 
stat = "YUF"
station_feature_dict, station_meta_dict, feature_names = proc.process_station_datasets(stat, 
                                                                                        all_train_df,
                                                                                        scaler=False,
                                                                                        linear_model=False,
                                                                                        source_dist_type='dist')

YUF
X shape: (1433, 45), y shape: (1433,)


In [5]:
feature_names

array(['amp_ratio_1', 'amp_ratio_2', 'amp_ratio_3', 'amp_ratio_4',
       'amp_ratio_5', 'amp_ratio_6', 'amp_ratio_7', 'amp_ratio_8',
       'amp_ratio_9', 'amp_ratio_10', 'amp_ratio_11', 'amp_ratio_12',
       'amp_ratio_13', 'amp_ratio_14', 'amp_ratio_15', 'amp_ratio_16',
       'amp_ratio_17', 'amp_ratio_18', 'amp_1', 'amp_2', 'amp_3', 'amp_4',
       'amp_5', 'amp_6', 'amp_7', 'amp_8', 'amp_9', 'amp_10', 'amp_11',
       'amp_12', 'amp_13', 'amp_14', 'amp_15', 'amp_16', 'amp_17',
       'amp_18', 'signal_dominant_frequency', 'signal_dominant_amplitude',
       'noise_max_amplitude', 'signal_max_amplitude', 'signal_variance',
       'noise_variance', 'source_depth_km',
       'source_receiver_distance_logkm',
       'source_receiver_back_azimuth_deg'], dtype='<U32')

# RFE to select N

Updates I need to make:

- RFECV is the outer loop
- Select N based on predictor model performance not estimator model performance
- tune predictor model hyperparameter for each N for each fold (optional?)
- Save the optimal N features from each fold 
- For each fold, do RFE with relative and absolute location information
- Save optimal N features (based on average performance for N over all folds) from the entire training set 
- Use MI to limit amplitudes and amplitude ratios (optional?)

In [6]:
def MI_filter_func(X, y, subsets, K, verbose=False):
    other_inds = np.arange(0, X.shape[1])[np.isin(np.arange(0, X.shape[1]), np.concatenate(subsets), invert=True)]
    filtered_inds = np.zeros((len(subsets), K), dtype=int)
    for i, subset in enumerate(subsets):
        Xi = X[:, subset]
        yi = y[:]
        filtered_inds[i, :] = subset[np.argsort(-1*ifs.mutual_reg_feature_selection(Xi, yi).scores_)[0:K]]
    new_inds = np.sort(np.concatenate([other_inds, filtered_inds.flatten()]))
    return filtered_inds, new_inds

In [7]:
### Set Parameters ###
cv_random_state=2652124
# The model to used to estimate feature importance 
estimator_model =  DecisionTreeRegressor()
# Boolean - True if data for estimator_model needs to be scaled
estimator_scaler = False
# The main model to fit
predictor_model = SVR(kernel='rbf')
# Boolean - True if data for model needs to be scaled 
model_scaler = True
# The scoring method to use in RFECV and GridCV
scoring_method = 'r2'
# The function used to evaluate the performance on the test model
score_func = r2_score
# True if a larger score_func value is better
larger_score_is_better = True
# The parameter space for GridCV to search
# Decrease upper-limit of C for linear model because 100 is very slow 
svr_Cspace = [1, 10, 100]
svr_gspace = [0.1, 0.01, 0.001]
param_grid = [
    {'m__C': svr_Cspace, 'm__gamma': svr_gspace},
]
# The number of jobs for RFECV and GridCV to use
n_jobs = 10
# number of folds for outer CV
cv_folds_outer = 10
# number of times to repeat outer CV
n_outer_repeats = 1
# number of folds for inner CV (used for hyperparameter tuning)
cv_folds_inner = 5
# number of folds for the final hyperparameter grid search
cv_folds_hp = 10
# Run grid search over all features
run_gridsearchcv_all = True

### Intrisic feature selection information 
# Function that takes in X, y, list of np arrays containing 
# the indices of features to filter, and K features to select
intrinsic_filter_func = MI_filter_func
if_feat_inds = [np.arange(0, 18), np.arange(18, 36)]
if_K = 5
#####################

In [8]:
N_feats_to_use, feat_selection_results = CustomRFECV.do_rfecv(station_feature_dict['X_train'],
                                                            station_meta_dict['y_train'],
                                                            estimator_model,
                                                            predictor_model,
                                                            param_grid,
                                                            estimator_scaler=estimator_scaler,
                                                            predictor_scaler=model_scaler,
                                                            score_func=score_func,
                                                            scoring_method=scoring_method,
                                                            n_jobs=n_jobs,
                                                            cv_folds_outer=cv_folds_outer,
                                                            cv_folds_inner=cv_folds_inner,
                                                            n_outer_repeats=n_outer_repeats,
                                                            cv_random_state=cv_random_state,
                                                            larger_score_is_better=larger_score_is_better,
                                                            intrinsic_filter_func=ifs.MI_filter_func,
                                                            feature_inds_to_filter=if_feat_inds,
                                                            intrinsic_filter_K=if_K
                                                            )

reducing features to 19
Fold 0: N=12, test_score=0.845
reducing features to 19
Fold 1: N=14, test_score=0.944
reducing features to 19
Fold 2: N=18, test_score=0.907
reducing features to 19
Fold 3: N=9, test_score=0.937
reducing features to 19
Fold 4: N=17, test_score=0.906
reducing features to 19
Fold 5: N=14, test_score=0.922
reducing features to 19
Fold 6: N=14, test_score=0.925
reducing features to 19
Fold 7: N=15, test_score=0.862
reducing features to 19
Fold 8: N=18, test_score=0.886
reducing features to 19
Fold 9: N=16, test_score=0.895
total time: 423.35 s
Selected number of features: 15 (avg. score of 0.90)


In [11]:
full_score_array = feat_selection_results['RFECV_N_scores']
N_avgs = np.average(full_score_array, axis=0)
N_min = np.min(full_score_array, axis=0)
N_max = np.max(full_score_array, axis=0)

In [12]:
x = np.arange(N_avgs.shape[0])+1
plt.fill_between(x, N_min, N_max, alpha=0.5)
plt.scatter(x, N_avgs, color='C0')
plt.scatter(N_feats_to_use, N_avgs[N_feats_to_use-1], marker="x", color='r')


# Get N features from entire training set (with same MI filtering and RFE with N features)